**Based on Peter Ferguson's notebook comparing EUCLID and ComCam data (/sdf/data/rubin/user/pferguso/projects/euclid/notebooks/euclid_star_compare.ipynb).**

**Best to use a LARGE container for the RSP notebook aspect.**

**Note that Euclid fluxes are in micro-Janskys; so mAB = -2.5log10(flux)+23.9** 
(See:  https://euclid.esac.esa.int/dr/q1/dpdd/merdpd/merphotometrycookbook.html)

## 1. User Input

See https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs for the latest values.

In [ ]:
# Path to the Euclid data Peter Ferguson downloaded to the USDF
#  (See https://rubin-obs.slack.com/archives/C07QM71RCUC/p1743131619724009 ) 
#euclid_path = "/home/p/pferguso/u/projects/euclid/q1/"      # original path
euclid_path = "/sdf/group/rubin/shared/euclid/q1/catalogs/"  # new path

repo="/repo/main"
collection="LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"
instrument = 'LSSTComCam'
skymap = 'lsst_cells_v1'

#outputFileName = 'euclid_stars_in_tract_2394.csv'   # ***OLD***
outputFileName = 'euclid_stars_in_tracts_2394_2234.csv'

#tractId = 2394   # ***OLD***
tract_list = [2394, 2234]
tract_dict={2394: 'EDFS', 2234: 'EDFS'}

## 2. Imports

In [ ]:
import copy

import pylab as plt
import numpy as np
import pandas as pd
from astropy.table import Table
import astropy.visualization as viz
import healpy as hp
import fitsio
from matplotlib.patches import Rectangle

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
from lsst import geom

afwDisplay.setDefaultBackend('matplotlib')
from smatch import match
import astropy.units as u

## 3. Useful functions

In [ ]:
# Useful class to stop "Run All" at a cell 
#  containing the command "raise StopExecution"
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [ ]:
zeropoint = 31.4 # AB zeropoint
def flux2mag(flux):
    return -2.5*np.log10(flux) + zeropoint

## 4.  Main code

In [ ]:
# Define which of the nearly 1000 columns to access from the ComCam ObjectTable...

INCOLS = [
    'coord_ra',
    'coord_dec',
    'detect_isPrimary',   
]
bands="griz"
for band in bands:
    INCOLS += [
        f'{band}_psfFlux',
        f'{band}_cModelFlux',
        f'{band}_psfFluxErr',
        f'{band}_extendedness',
        f'{band}_psfFlux_flag'
    ]

In [ ]:
## Access the ComCam data.  ***OLD***
## This is just to find what healpixel regions to grab from the Euclid data that Peter Ferguson downloaded to USDF.
#
#butler = dafButler.Butler(repo, collections=collection)
#
#raw_comcam = butler.get('objectTable_tract', dataId={'skymap': 'lsst_cells_v1', 'tract': tractId}, collections=[collection],
#                        parameters={"columns":INCOLS})
## Clean the catalog
#sel  = (raw_comcam['detect_isPrimary'] == True)
#sel &= (raw_comcam['r_psfFlux']/raw_comcam['r_psfFluxErr'] > 5)
#for band in bands:
#    sel &= (raw_comcam[f'{band}_psfFlux_flag'] == 0)
#comcam = raw_comcam[sel]
#
#sel_comcam_stars = (comcam['g_extendedness'] < 0.5) & (comcam['r_extendedness'] < 0.5)
#RA,DEC = comcam['coord_ra'].median(), comcam['coord_dec'].median()
#pix = hp.ang2pix(64, RA, DEC, lonlat=True, nest=True)
#upix = np.unique(hp.ang2pix(64, comcam['coord_ra'], comcam['coord_dec'], lonlat=True, nest=True))
#print(f"NSIDE=64, Nest, HEALPix: {pix}")
#
#print(f"Number of objects: {len(comcam)}")
#print(f"Number of stars: {sel_comcam_stars.sum()}")

In [ ]:
# Access the ComCam data.
# This is just to find what healpixel regions to grab from the Euclid data that Peter Ferguson downloaded to USDF.

butler = dafButler.Butler(repo, collections=collection)

comcam_list = []

for tractId in tract_list:

    print(tractId, tract_dict[tractId])

    try:
    
        raw_comcam = butler.get('objectTable_tract', dataId={'skymap': 'lsst_cells_v1', 'tract': tractId}, 
                                collections=[collection],
                                parameters={"columns":INCOLS})

        # Insert tractId as the first column
        raw_comcam.insert(0, 'tractId', tractId)  
    
        # Insert field name -- if known -- as the second column
        if tractId in tract_dict:
            field = tract_dict[tractId]
        else:
            field = 'unknown'
        raw_comcam.insert(1, 'field', field)  

        # Clean the catalog
        sel  = (raw_comcam['detect_isPrimary'] == True)
        sel &= (raw_comcam['r_psfFlux']/raw_comcam['r_psfFluxErr'] > 5)
#        for band in bands:
        for band in ['g','r','i']:
            sel &= (raw_comcam[f'{band}_psfFlux_flag'] == 0)
        comcam = raw_comcam[sel]
        
        # Append the dataframe to the list
        comcam_list.append(comcam)

    # Catch any exception
    except Exception as e:

        print(f"An error occurred for tractId {tractId}: {e}")

# Concatenate all dataframes in the list
comcam_all = pd.concat(comcam_list, ignore_index=True)  
print(f"Total number of stars: {len(comcam_all)}")

# Find just the (most likely) stars...
sel_comcam_all_stars = (comcam_all['g_extendedness'] < 0.5) & (comcam_all['r_extendedness'] < 0.5)
RA,DEC = comcam_all['coord_ra'].median(), comcam_all['coord_dec'].median()
pix = hp.ang2pix(64, RA, DEC, lonlat=True, nest=True)
upix = np.unique(hp.ang2pix(64, comcam_all['coord_ra'], comcam_all['coord_dec'], lonlat=True, nest=True))
print(f"NSIDE=64, Nest, HEALPix: {pix}")

print(f"Number of objects: {len(comcam_all)}")
print(f"Number of stars: {sel_comcam_all_stars.sum()}")

In [ ]:
# sky coverage of these ComCam data in the form of a list of NSIDE=64 Nest Healpix pixels: 
upix

In [ ]:
# Access the euclid data
euclid = []
for pix in upix:
    euclid.append(fitsio.read(euclid_path + f"euclid_q1_mer_final_{pix:05d}.fits"))
euclid = np.concatenate(euclid)

# Clean and restrict the catalog
region  = (euclid['RIGHT_ASCENSION'] > comcam_all['coord_ra'].min()) & (euclid['RIGHT_ASCENSION'] < comcam_all['coord_ra'].max())
region &= (euclid['DECLINATION'] > comcam_all['coord_dec'].min()) & (euclid['DECLINATION'] < comcam_all['coord_dec'].max())

nside=4096
comcam_all_hpx = hp.ang2pix(nside, comcam_all['coord_ra'], comcam_all['coord_dec'], lonlat=True)
comcam_all_uid, comcam_all_cts = np.unique(comcam_all_hpx, return_counts=True)
euclid_hpx = hp.ang2pix(nside, euclid['RIGHT_ASCENSION'], euclid['DECLINATION'], lonlat=True)

sel  = (euclid['FLUX_VIS_PSF'] / euclid['FLUXERR_VIS_PSF'] > 5)
sel &= (euclid['SPURIOUS_FLAG'] == 0) 
ebands = ['VIS', 'Y', 'J', 'H']
for eband in ebands:
    sel &= (euclid[f'FLAG_{eband}'] == 0)
sel &= ((euclid['DET_QUALITY_FLAG'] == 0) | (euclid['DET_QUALITY_FLAG'] == 2) | (euclid['DET_QUALITY_FLAG'] == 512))
sel &= region
sel &= np.in1d(euclid_hpx, comcam_all_uid[comcam_all_cts > 8])
euclid = euclid[sel]
print(f"N good obj: {sel.sum()}")
sel_euclid_stars = (euclid["POINT_LIKE_PROB"] > 0.7)
print(f"N stars: {sel_euclid_stars.sum()}")

In [ ]:
# Create an Astropy Table for the Euclid stars from the array data returned in the previous cell...

t_euclid_stars = Table(euclid[sel_euclid_stars])

In [ ]:
# Creat a Pandas DataFrame from the Astropy Table of Euclid stars...

df_euclid_stars = t_euclid_stars.to_pandas()

In [ ]:
# Define a set of useful Euclid columns to output to a CSV file

euclid_outcols = ['OBJECT_ID','RIGHT_ASCENSION','DECLINATION',
                 'FLUX_VIS_PSF','FLUXERR_VIS_PSF',
                 'FLUX_VIS_1FWHM_APER', 'FLUXERR_VIS_1FWHM_APER',
                 'FLUX_VIS_2FWHM_APER', 'FLUXERR_VIS_2FWHM_APER',
                 'FLUX_VIS_3FWHM_APER', 'FLUXERR_VIS_3FWHM_APER',
                 'FLUX_VIS_4FWHM_APER', 'FLUXERR_VIS_4FWHM_APER',
                 'FLUX_Y_1FWHM_APER', 'FLUXERR_Y_1FWHM_APER',
                 'FLUX_Y_2FWHM_APER', 'FLUXERR_Y_2FWHM_APER',
                 'FLUX_Y_3FWHM_APER', 'FLUXERR_Y_3FWHM_APER',
                 'FLUX_Y_4FWHM_APER', 'FLUXERR_Y_4FWHM_APER',
                 'FLUX_J_1FWHM_APER', 'FLUXERR_J_1FWHM_APER',
                 'FLUX_J_2FWHM_APER', 'FLUXERR_J_2FWHM_APER',
                 'FLUX_J_3FWHM_APER', 'FLUXERR_J_3FWHM_APER',
                 'FLUX_J_4FWHM_APER', 'FLUXERR_J_4FWHM_APER',
                 'FLUX_H_1FWHM_APER', 'FLUXERR_H_1FWHM_APER',
                 'FLUX_H_2FWHM_APER', 'FLUXERR_H_2FWHM_APER',
                 'FLUX_H_3FWHM_APER', 'FLUXERR_H_3FWHM_APER',
                 'FLUX_H_4FWHM_APER', 'FLUXERR_H_4FWHM_APER'
                ]

In [ ]:
# Output the columns to the screen...

df_euclid_stars[euclid_outcols]

In [ ]:
# Output results to a CSV file...

df_euclid_stars[euclid_outcols].to_csv(outputFileName,index=False)

**Remember that Euclid fluxes are in micro-Janskys; so mAB = -2.5log10(flux)+23.9** 
(See:  https://euclid.esac.esa.int/dr/q1/dpdd/merdpd/merphotometrycookbook.html)